# GluonNLP の BERT モデル を利用した感情分析

## 概要

このノートブックでは、Amazon の商品レビューに対する感情分析、つまり、そのレビューが Positive (Rating が 5 or 4) か、Negative (Rating が 1 or 2)なのかを判定します。これは文書を Positive か Negative に分類する2クラスの分類問題となります。そこで、BERT (Bidirectional Encoder Representations from Transformers) モデルを利用して解きます。

### BERT とは

BERT は大規模なコーパスで学習された汎用的な自然言語処理のためのモデルです。今回対象とする文書の分類問題だけでなく、文書のペアを分類するような質問応答の問題に対しても、転移学習を行うことで良い精度を示しています。BERT は大規模なモデルであり、学習には多くの時間が必要です。

### GluonNLPとは
MXNet をより簡単に利用するためのライブラリとして Gluon が開発されています。Gluon には、自然言語処理に特化した GluonNLP という派生のライブラリがあります。その中には、BERT モデルの学習済みモデルも提供されており、BERT の事前の学習時間の削減や、実装の効率化に有効です。詳細はURLをごらんください。

https://gluon-nlp.mxnet.io/index.html


## データの準備

Amazon の商品レビューデータセットは [Registry of Open Data on AWS](https://registry.opendata.aws/) で公開されており、 
以下からダウンロード可能です。このノートブックでは、日本語のデータセットをダウンロードします。
- データセットの概要  
https://registry.opendata.aws/amazon-reviews/

- 日本語のデータセット(readme.htmlからたどることができます）  
https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_JP_v1_00.tsv.gz

以下では、データをダウンロードして解凍 (unzip) します。

In [ ]:
import urllib.request
import os
import gzip
import shutil

download_url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_JP_v1_00.tsv.gz" 
dir_name = "data"
file_name = "amazon_review.tsv.gz"
tsv_file_name = "amazon_review.tsv"
file_path = os.path.join(dir_name,file_name)
tsv_file_path = os.path.join(dir_name,tsv_file_name)

os.makedirs(dir_name, exist_ok=True)

if os.path.exists(file_path):
    print("File {} already exists. Skipped download.".format(file_name))
else:
    urllib.request.urlretrieve(download_url, file_path)
    print("File downloaded: {}".format(file_path))
    
if os.path.exists(tsv_file_path):
    print("File {} already exists. Skipped unzip.".format(tsv_file_name))
else:
    with gzip.open(file_path, mode='rb') as fin:
        with open(tsv_file_path, 'wb') as fout:
            shutil.copyfileobj(fin, fout)
            print("File uznipped: {}".format(tsv_file_path))

## データの前処理

ダウンロードしたデータには学習に不要なデータや直接利用できないデータもあります。以下の前処理で利用できるようにします。

1. ダウンロードしたデータには不要なデータも含まれているので削除し、2クラス分類 (positive が 1, negative が 0)となるように評価データを加工します。
2. 学習データ、テストデータに分けて、学習用にS3にデータをアップロードします。

### データの加工

今回利用しないデータは以下の2つです。必要なデータだけ選んで保存します。
- 評価データ `star_rating` と レビューのテキストデータ `review_body` 以外のデータ
- 評価が 3 のデータ (positive でも negative でもないデータ)

また、評価が1, 2 のデータはラベル 0 (negative) に、評価が4, 5 のデータはラベル 1 (positive) にします。BERTには、Tokenizer という分かち書きのための機能が備わっています。従って、学習を実行する直前に分かち書きを行うようにし、ここでは行いません。

In [ ]:
import pandas as pd
df = pd.read_csv(tsv_file_path, sep ='\t')
df_pos_neg = df.loc[:, ["star_rating", "review_body"]]
df_pos_neg = df_pos_neg[df_pos_neg.star_rating != 3]
df_pos_neg.loc[df_pos_neg.star_rating < 3, "star_rating"] = 0
df_pos_neg.loc[df_pos_neg.star_rating > 3, "star_rating"] = 1

### データの分割

すべてのデータを学習データとすると、データを使って作成したモデルが良いのか悪いのか評価するデータが別途必要になります。
そこで、データを学習データ、テストデータに分割して利用します。学習データはモデルの学習に利用し、最終的に作成されたモデルに対してテストデータによる評価を行います。

`train_ratio` で設定した割合のデータを学習データとし、残ったデータをテストデータに分割して利用します。学習データは、後にSageMakerで利用するために、`savetxt` を利用してスペース区切りの csv に保存します。

In [ ]:
import numpy as np

# Swap positions of "review_body","star_rating" because transform.py requires this order.
labeled_df = df_pos_neg.loc[:, ["review_body","star_rating"]]
data_size = len(labeled_df.index)
train_ratio = 0.9
train_index = np.random.choice(data_size, int(data_size*train_ratio), replace=False)
test_index = np.setdiff1d(np.arange(data_size), train_index)

np.savetxt('train.tsv',labeled_df.iloc[train_index].values, fmt="%s\t%i") 

print("Data is splitted into:")
print("Training data: {} records.".format(len(train_index)))
print("Test data: {} records.".format(len(test_index)))

### データのアップロード

SageMaker での学習に利用するために、学習データを S3 にアップロードします。SageMaker Python SDK の upload_data を利用すると、S3 にファイルをアップロードできます。アップロード先のバケットは `sagemaker-{リージョン名}-{アカウントID}`で、バケットがない場合は自動作成されます。もし存在するバケットにアップロードする場合は、このバケット名を引数で指定できます。

In [ ]:
import sagemaker

sess = sagemaker.Session()

s3_train_data = sess.upload_data(path='train.tsv', key_prefix='amazon-review-data')
print("Training data is uploaded to {}".format(s3_train_data))

data_channels = {'train': s3_train_data}

## 学習の実行

### 学習コードの作成

GluonNLP を利用した BERT の学習とデプロイ用のコードを `train_and_deploy.py` として作成します。コード作成にあたっては、GluonNLP の公式ページのチュートリアルのコードを流用可能です。

https://gluon-nlp.mxnet.io/examples/sentence_embedding/bert.html

ただし、この公式チュートリアルはペアの文書を分類する前提になっています。今回はペアを利用しないため、以下の点を修正して利用します。

```python
pair = False
```

### GluonNLP のインストール

SageMaker のコンテナでは、mxnet 1.6.0のバージョンから GluonNLP がデフォルトでインストールされています。このため、GluonNLP をインストールするためのスクリプトは不要です。

### 学習ジョブの実行

MXNet のコンテナを呼び出し、学習用のインスタンスを指定して、学習を実行します。`hyperparameters`で渡すパラメータは train_and_deploy.py でパースして利用することができます。BERTのモデルの学習は非常に時間がかかるため、1エポックだけ回すようにします。それでも20分以上かかりますのでご注意ください。

fit()を指定すれば、S3のデータを渡して学習することが可能です。

In [ ]:
from sagemaker.mxnet import MXNet

gluon_bert = MXNet("train_and_deploy.py", 
                  role=sagemaker.get_execution_role(), 
                  source_dir = "src",
                  instance_count=1, 
                  instance_type="ml.m4.xlarge",
                  framework_version="1.6.0",
                  distribution={'parameter_server': {'enabled': True}},
                  py_version = "py3",
                  hyperparameters={'batch-size': 16, 
                                   'epochs': 1, 
                                   'log-interval': 1})

In [ ]:
gluon_bert.fit(data_channels)

## 推論の実行

学習が終わると、作成されたモデルをデプロイして、推論を実行することができます。デプロイは deploy を呼び出すだけでできます。`---`といった出力があるときはデプロイ中で、`!`が出力されるとデプロイが完了です。

### 推論のコード

SageMakerがサポートしている機械学習フレームワークコンテナで推論を行う場合は、モデルの読み込みや前処理・後処理を容易に実装できます。MXNet の場合は、モデル読み込みに `model_fn`、前処理・後処理に `transform_fn` を実装します。model_fnでは学習したモデルだけでなく、Tokenizerも読み込んでおき、`transformer_fn`で利用します。

```python
def model_fn(model_dir):
    bert_tokenizer = nlp.data.BERTTokenizer(vocabulary, lower=True)
    bert_classifier = gluon.SymbolBlock.imports(
        '%s/model-symbol.json' % model_dir,
        ['data0', 'data1', 'data2'],
        '%s/model-0000.params' % model_dir,
    )
    return {"net": bert_classifier, "tokenizer": bert_tokenizer}

def transform_fn(net, data, input_content_type, output_content_type):
 
    bert_classifier = net["net"]
    bert_tokenizer = net["tokenizer"]
 
    parsed = json.loads(data)
    logging.info("Received_data: {}".format(parsed))
    tokens = bert_tokenizer(parsed)
    logging.info("Tokens: {}".format(tokens))
    token_ids = bert_tokenizer.convert_tokens_to_ids(tokens)
    valid_length = len(token_ids)
    segment_ids = mx.nd.zeros([1, valid_length])

    output = bert_classifier(mx.nd.array([token_ids]), 
                             segment_ids, 
                             mx.nd.array([valid_length]).astype('float32'))
    response_body = json.dumps(output.asnumpy().tolist()[0])
    return response_body, output_content_type
```


In [ ]:
predictor = gluon_bert.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

デプロイが終わったら推論を実行してみましょう。ここでは negative なレビューを 5件、 positive なレビューを 5件ランダムに選択して推論を実行します。1エポックしか実行しない場合、良い精度を得ることは難しいと思います。ぜひ多くのエポックを試してみてください。

In [ ]:
import mxnet as mx

num_test = 5
test_data = labeled_df.iloc[test_index]

neg_test_data = test_data[test_data.star_rating == 0]
pos_test_data = test_data[test_data.star_rating == 1]

neg_index = np.random.choice(neg_test_data.index, num_test)
pos_index = np.random.choice(pos_test_data.index, num_test)

for i in neg_index:
    pred = predictor.predict(neg_test_data.loc[i, "review_body"])
    prob = mx.nd.softmax(mx.nd.array(pred).reshape([1,2]))
    pred_label =np.argmax(prob, axis = 1)
    print("Ground Truth: {}, Prediction: {} (probability: {})"
                      .format(0, pred_label.asscalar(), prob[0, pred_label[0]].asscalar()))
    print(neg_test_data.loc[i, "review_body"])
    print()
    

for i in pos_index:
    pred = predictor.predict(pos_test_data.loc[i, "review_body"])
    prob = mx.nd.softmax(mx.nd.array(pred).reshape([1,2]))
    pred_label =np.argmax(prob, axis = 1)
    print("Ground Truth: {}, Prediction: {} (probability: {})"
                      .format(1, pred_label.asscalar(), prob[0, pred_label[0]].asscalar()))
    print(pos_test_data.loc[i, "review_body"])
    print()

### 不要になったエンドポイントを削除

In [ ]:
predictor.delete_endpoint()